# Morphing basis animations

Let's make something cool:

![morphing animation](morphing.gif)

In [5]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib inline

from madminer.utils.morphing import PhysicsMorpher as Morpher

## Settings

In [6]:
n_runs = 10
n_resolution = 200
n_frames = 8 * 25

randomness1 = 0.3
randomness2 = 0.6

## Preparation

In [7]:
morpher = Morpher(
    parameter_max_power=[4, 4],
    parameter_range=[(-1.0, 1.0), (-1.0, 1.0)],
)
morpher.find_components(max_overall_power=4)

In [8]:
xi = np.linspace(-1.0, 1.0, n_resolution)
yi = np.linspace(-1.0, 1.0, n_resolution)
xx, yy = np.meshgrid(xi, yi)
xx = xx.reshape((-1, 1))
yy = yy.reshape((-1, 1))
theta_evaluation = np.hstack([xx, yy])

## Basis evaluation

In [9]:
def evaluate_basis(basis=None):

    # Optimization
    if basis is None:
        basis = morpher.optimize_basis(n_bases=1)

    # Evaluate basis
    squared_weights = []

    for theta in theta_evaluation:
        wi = morpher.calculate_morphing_weights(theta, basis)
        squared_weights.append(np.sum(wi * wi) ** 0.5)

    squared_weights = np.array(squared_weights).reshape((n_resolution, n_resolution))

    return squared_weights

## Basis point trajectories

In [10]:
def initialize_trajectories():
    global theta1, theta2, theta_phase, theta_order

    # fmt: off
    theta_fix = np.array([
       [ 0.0       ,  0.0       ],
       [-0.83182450,  0.85645093],
       [-0.82002127, -0.85191237],
       [ 0.76870769, -0.81272456],
       [ 0.78199620,  0.86242685],
       [-0.57243257,  0.37755934],
       [-0.29730939,  0.74563426],
       [ 0.13777926,  0.35254704],
       [ 0.46330191,  0.51783982],
       [ 0.64649576, -0.01232633],
       [ 0.16629182, -0.29365045],
       [ 0.39752054, -0.64235507],
       [-0.19238158, -0.59962178],
       [-0.30730345, -0.09697784],
       [-0.70631846, -0.18913046],
    ])
    # fmt: on

    n_benchmarks = theta_fix.shape[0]

    theta_order = np.random.randint(1, 4, (n_benchmarks, 2))

    random_points = -1.0 + 2.0 * np.random.rand(n_benchmarks, 2)
    random_point_weights = randomness1 * np.random.rand(n_benchmarks, 2) / theta_order.astype(float)
    theta1 = (1.0 - random_point_weights) * theta_fix + random_point_weights * random_points

    random_points = -1.0 + 2.0 * np.random.rand(n_benchmarks, 2)
    random_point_weights = randomness2 * np.random.rand(n_benchmarks, 2) / theta_order.astype(float)
    theta2 = (1.0 - random_point_weights) * theta_fix + random_point_weights * random_points

    theta_phase = 2.0 * np.pi * np.random.rand(n_benchmarks, 2)

In [11]:
def calculate_basis(t):
    basis = 0.5 * (
        (theta1 + theta2) + (theta1 - theta2) * np.sin(2.0 * np.pi * theta_order * t / n_frames + theta_phase)
    )
    return basis

## Animation

In [12]:
def initialize_animation():
    global basis, squared_weights

    initialize_trajectories()
    basis = calculate_basis(0)
    squared_weights = evaluate_basis(basis)

In [13]:
def update(t):
    global basis, squared_weights

    basis = calculate_basis(t)
    squared_weights = evaluate_basis(basis)

    scatter.set_offsets(basis)
    pcm.set_array(squared_weights[:-1, :-1].ravel())

    if (t + 1) % 10 == 0:
        print("  Frame", t + 1, "/", n_frames)

In [14]:
def make_animation(i):
    global pcm, scatter

    fig = plt.figure(figsize=(5.0, 4.0))
    ax = plt.gca()

    pcm = ax.pcolormesh(
        xi,
        yi,
        squared_weights,
        norm=matplotlib.colors.LogNorm(vmin=0.5, vmax=200.0),
        cmap="viridis_r",
    )
    cbar = fig.colorbar(pcm, ax=ax, extend="both")

    scatter = plt.scatter(basis[:, 0], basis[:, 1], s=40.0, c="black")

    plt.xlabel(r"$\theta_0$")
    plt.ylabel(r"$\theta_1$")
    cbar.set_label(r"$\sqrt{\sum w_i^2}$")
    plt.xlim(-1.0, 1.0)
    plt.ylim(-1.0, 1.0)

    plt.tight_layout()

    anim = FuncAnimation(fig, update, frames=np.arange(0, n_frames), interval=40)
    anim.save(f"animations/run_{i}.gif", dpi=120, writer="imagemagick")

## Main loop

In [ ]:
for i in range(n_runs):
    print(f"Run {i+1} / {n_runs}")
    initialize_animation()
    make_animation(i)